<a href="https://colab.research.google.com/github/Arhodes-Broad/GATK_Workshop_Materials/blob/master/gatk_hard_filtering_taiwan_20181125.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GATK Tutorial | Hard Filtering | November 2018

This GATK tutorial corresponds to a section of the GATK Workshop _2b. Germline Hard Filtering Tutorial_ worksheet available at [https://drive.google.com/open?id=1HeA-zuPW-uEZCJP5zd3AAYfOdwgIWUld](https://drive.google.com/open?id=1HeA-zuPW-uEZCJP5zd3AAYfOdwgIWUld). The goal is to become familiar with germline variant annotations. The notebook illustrates the following steps. 

- Use GATK to stratify a variant callset against a truthset
- Use R's ggplot2 package to plot the distribution of various annotation values
- Hard-filter based on annotation thresholds and calculate concordance metrics  

### First, make sure the notebook is using a Python 3 kernel in the top right corner.
The notebook switches between using Python 3 and R kernels. A kernel is a _computational engine_ that executes the code in the notebook. We have to switch kernels so that we can execute GATK commands using _Python Magic_ (`!`) and make plots using R. 

### How to run this notebook:
- **Click to select a gray cell and then pressing SHIFT+ENTER to run the cell.**
- **When instructions say to change kernels, go to the menubar and select _Kernel > Change Kernel > Python 3_.** When switching from R back to Python, any R variables will be lost.

- **Write results to `/home/jupyter-user/`. To access the directory, click on the upper-left jupyter icon.**

### Enable reading Google bucket data 

In [0]:
# Check if data is accessible. The command should list several gs:// URLs.
! gsutil ls gs://gatk-tutorials/workshop_1702/variant_discovery/data/resources/
! gsutil ls gs://gatk-tutorials/workshop_1702/variant_discovery/data/intervals/motherHighconf.bed
! gsutil ls gs://gatk-tutorials/workshop_1702/variant_discovery/data/inputVcfs/

In [0]:
# If you do not see gs:// URLs listed above, run this cell to install Google Cloud Storage. 
# Afterwards, restart the kernel with Kernel > Restart.
! pip install google-cloud-storage

---
## 1. Subset variants to SNPs of a single sample with SelectVariants

Subset the trio callset to just the SNPs of the mother (sample NA12878). Make sure to remove sites for which the sample genotype is homozygous-reference and remove unused alleles, including spanning deletions. 

> The tool recalculates depth of coverage (DP) per site as well as the allele count in genotypes for each ALT allele (AC), allele frequency for each ALT allele (AF), and  total number of alleles in called genotypes (AN), to reflect only the subset sample(s).

In [0]:
! gatk SelectVariants \
-V gs://gatk-tutorials/workshop_1702/variant_discovery/data/inputVcfs/trio.vcf.gz \
-sn NA12878 \
-select-type SNP \
--exclude-non-variants \
--remove-unused-alternates \
-O /home/jupyter-user/motherSNP.vcf.gz

In [0]:
# Peruse the resulting file 
! zcat /home/jupyter-user/motherSNP.vcf.gz | grep -v '##' | head

---
## 2. Annotate intersecting true positives with VariantAnnotator

We use VariantAnnotator to annotate which variants in our callset are also present in the truthset (GIAB), which are considered true positives. Variants not present in the truthset are considered false positives. Here we produce a callset where variants that are present in the truthset are annotated with the giab.callsets annotation plus a value indicating how many of the callsets used to develop the truthset agreed with that call.

In [0]:
! gatk VariantAnnotator \
-V /home/jupyter-user/motherSNP.vcf.gz \
--resource giab:gs://gatk-tutorials/workshop_1702/variant_discovery/data/resources/motherGIABsnps.vcf.gz \
-E giab.callsets \
-O /home/jupyter-user/motherSNP.giab.vcf.gz

In [0]:
# Peruse the resulting file 
! zcat /home/jupyter-user/motherSNP.giab.vcf.gz | grep 'giab.callsets' | head

---
## 3. Tabulate annotations of interest with VariantsToTable

Convert the information from the callset into a table using VariantsToTable, so that we can parse it easily in R. The tool parameters differentiate cohort-level fields (`-F`) and sample-level genotype fields (`-GF`). This step produces a table where each line represents a variant record from the VCF, and each column represents an annotation we have specified. Wherever the requested annotations are not present, e.g. RankSum annotations at homozygous sites, the value will be replaced by NA. 

In [0]:
! gatk VariantsToTable \
-V /home/jupyter-user/motherSNP.giab.vcf.gz \
-F CHROM -F POS -F QUAL \
-F BaseQRankSum -F MQRankSum -F ReadPosRankSum \
-F DP -F FS -F MQ -F QD -F SOR \
-F giab.callsets \
-GF GQ \
-O /home/jupyter-user/motherSNP.giab.txt

In [0]:
# Peruse the resulting file
! cat /home/jupyter-user/motherSNP.giab.txt | head -n300

In [0]:
# Focus in on a few columns
! cat /home/jupyter-user/motherSNP.giab.txt | cut -f1,2,7,12 | head -n300

---
## 4. Make density and scatter plots in R and determine filtering thresholds

<span style="color:red">Change the kernel to R. Go to the menubar and select _Kernel > Change Kernel > R_.</span>

Plotting the density of values for an annotation shows us to see the overall range and distribution of values observed in a callset. In combination with some basic knowledge of what the annotation represents and how it is calculated, this allows us to make a first estimation of value thresholds that segregate FPs from TPs. Plotting the scatter of values for two annotations, one against the other, additionally shows us what tradeoffs we make when setting a threshold on annotation values individually. 

### A. Load R libraries, plotting functions and data

Don't worry if you don't know how to read the R script below. Also, you can ignore the red boxes that appear, e.g. stating `as ‘lib’ is unspecified`. 

In [0]:
# plotting.R script loads ggplot and gridExtra libraries and defines functions to plot variant annotations 

library(ggplot2)
install.packages("gridExtra")
library(gridExtra)

get_legend<-function(myggplot){
  tmp <- ggplot_gtable(ggplot_build(myggplot))
  leg <- which(sapply(tmp$grobs, function(x) x$name) == "guide-box")
  legend <- tmp$grobs[[leg]]
  return(legend)
}


# Function for making density plots of a single annotation
makeDensityPlot <- function(dataframe, xvar, split, xmin=min(dataframe[xvar], na.rm=TRUE), xmax=max(dataframe[xvar], na.rm=TRUE), alpha=0.5) {
  
  if(missing(split)) {
    return(ggplot(data=dataframe, aes_string(x=xvar)) + xlim(xmin,xmax) + geom_density() )
  }
  else {
    return(ggplot(data=dataframe, aes_string(x=xvar, fill=split)) + xlim(xmin,xmax) + geom_density(alpha=alpha) )
  }
}

# Function for making scatter plots of two annotations
makeScatterPlot <- function(dataframe, xvar, yvar, split, xmin=min(dataframe[xvar], na.rm=TRUE), xmax=max(dataframe[xvar], na.rm=TRUE), ymin=min(dataframe[yvar], na.rm=TRUE), ymax=max(dataframe[yvar], na.rm=TRUE), ptSize=1, alpha=0.6) {
  if(missing(split)) {
    return(ggplot(data=dataframe) + aes_string(x=xvar, y=yvar) + xlim(xmin,xmax) + ylim(ymin,ymax) + geom_point(size=ptSize, alpha=alpha) )
  }
  else {
    return(ggplot(data=dataframe) + aes_string(x=xvar, y=yvar) + aes_string(color=split) + xlim(xmin,xmax) + ylim(ymin,ymax) + geom_point(size=ptSize, alpha=alpha) )
  }
}

# Function for making scatter plots of two annotations with marginal density plots of each
makeScatterPlotWithMarginalDensity <- function(dataframe, xvar, yvar, split, xmin=min(dataframe[xvar], na.rm=TRUE), xmax=max(dataframe[xvar], na.rm=TRUE), ymin=min(dataframe[yvar], na.rm=TRUE), ymax=max(dataframe[yvar], na.rm=TRUE), ptSize=1, ptAlpha=0.6, fillAlpha=0.5) {
  empty <- ggplot()+geom_point(aes(1,1), colour="white") +
    theme(
      plot.background = element_blank(), 
      panel.grid.major = element_blank(), 
      panel.grid.minor = element_blank(), 
      panel.border = element_blank(), 
      panel.background = element_blank(),
      axis.title.x = element_blank(),
      axis.title.y = element_blank(),
      axis.text.x = element_blank(),
      axis.text.y = element_blank(),
      axis.ticks = element_blank()
    )
  
  if(missing(split)){
    scatter <- ggplot(data=dataframe) + aes_string(x=xvar, y=yvar) + geom_point(size=ptSize, alpha=ptAlpha) + xlim(xmin,xmax) + ylim(ymin,ymax) 
    plot_top <- ggplot(data=dataframe, aes_string(x=xvar)) + geom_density(alpha=fillAlpha) + theme(legend.position="none") + xlim(xmin,xmax) 
    plot_right <- ggplot(data=dataframe, aes_string(x=yvar)) + geom_density(alpha=fillAlpha) + coord_flip() + theme(legend.position="none") + xlim(ymin,ymax) 
  } 
  else{
    scatter <- ggplot(data=dataframe) + aes_string(x=xvar, y=yvar) + geom_point(size=ptSize, alpha=ptAlpha, aes_string(color=split)) + xlim(xmin,xmax) + ylim(ymin,ymax) 
    plot_top <- ggplot(data=dataframe, aes_string(x=xvar, fill=split)) + geom_density(alpha=fillAlpha) + theme(legend.position="none") + xlim(xmin,xmax) 
    plot_right <- ggplot(data=dataframe, aes_string(x=yvar, fill=split)) + geom_density(alpha=fillAlpha) + coord_flip() + theme(legend.position="none") + xlim(ymin,ymax) 
  }
  legend <- get_legend(scatter)
  scatter <- scatter + theme(legend.position="none")
  temp <- grid.arrange(plot_top, legend, scatter, plot_right, ncol=2, nrow=2, widths=c(4,1), heights=c(1,4))
  return(temp)
}

In [0]:
# Call the readr library and use its read_delim function to load motherSNP.giab.txt into the motherSNP.giab object.
library(readr)
motherSNP.giab <- read_delim("/home/jupyter-user/motherSNP.giab.txt","\t", 
              escape_double = FALSE, col_types = cols(giab.callsets = col_character()), trim_ws = TRUE)

In [0]:
# Rename the 'giab.callsets' column to 'set'.
names(motherSNP.giab)[names(motherSNP.giab) == 'giab.callsets'] <- 'set'

---

For reference, here are some basic filtering thresholds to improve upon.

- -filter "QD < 2.0"
- -filter "QUAL < 30.0"
- -filter "SOR > 3.0"
- -filter "FS > 60.0"
- -filter "MQ < 40.0"
- -filter "MQRankSum < -12.5 
- -filter "ReadPosRankSum < -8.0"

---

### B. Make a density plot for QUAL with the `makeDensityPlot` function

Iteratively improve the plot by modifying `qual`. Here are some suggestions to start.
- B = makeDensityPlot(motherSNP.giab, "QUAL")
- B = makeDensityPlot(motherSNP.giab, "QUAL", xmax=10000)
- B = makeDensityPlot(motherSNP.giab, "QUAL", xmax=10000, split="set")

> _How does the density distribution relate to what the annotation represents? Can we find some clues of what might distinguish good vs. bad variants?_
> _When we plot the split version, can we see a clear difference between the set distributions? What does that tell us?_

In [0]:
# B = makeDensityPlot(motherSNP.giab, "QUAL")
B = makeDensityPlot(motherSNP.giab, 
                    "QUAL")

In [0]:
# Plot 'B'
B

### C. Make a QD (QualByDepth) density plot

QD puts the variant confidence QUAL score into perspective by normalizing for the amount of coverage available. Because each read contributes a little to the QUAL score, variants in regions with deep coverage can have artificially inflated QUAL scores, giving the impression that the call is supported by more evidence than it really is. To compensate for this, we normalize the variant confidence by depth, which gives us a more objective picture of how well supported the call is.

> _What do the peaks represent?_

In [0]:
# C = makeDensityPlot(motherSNP.giab, "QD")
# Change up the parameters, e.g. add 'split="set"', examine RankSums, FS and SOR
C = makeDensityPlot(motherSNP.giab, 
                    "QD")

In [0]:
C

### D. Make a scatterplot of QD vs. DP using the `makeScatterPlot` function

The DP (depth) here refers to the unfiltered count of reads at the site level (INFO). An identically named annotation exists at the sample level (FORMAT) that refers to the count of reads that passed the caller's internal quality control metrics for the sample. 

> What is the relationship between DP and QUAL? How does high-depth correlate with true positives?

In [0]:
# D = makeScatterPlot(motherSNP.giab, "QD", "DP", split="set")
# Play with the axis limits to zoom in on subsets of the data, e.g. by adding ymax=1000.
D = makeScatterPlot(motherSNP.giab, 
                    "QD", "DP", 
                    split="set")

In [0]:
D

### E. Make a scatterplot winged by marginal density plots

The `makeScatterPlotWithMarginalDensity` function defines and plots. The `ptAlpha` parameter changes the transparency of the points. 

> _When plotting two annotations, does the combination of the two tell us anything more than either did separately?_

- Try adjusting the parameters.
- Substitute in other annotations. For example, the following recreates the plot on the front page of the tutorial worksheet.

```
F = makeScatterPlotWithMarginalDensity(motherSNP.giab, "QUAL", "DP", split="set", xmax=10000, ymax=100, ptSize=0.5, ptAlpha=0.05)
```




In [0]:
# E = makeScatterPlotWithMarginalDensity(motherSNP.giab, "QD", "DP", split="set", ymax=250, ptSize=0.5, ptAlpha=0.2)
E = makeScatterPlotWithMarginalDensity(motherSNP.giab, 
                                       "QD", "DP", 
                                       split="set", 
                                       ymax=250, 
                                       ptSize=0.5, ptAlpha=0.2)

In [0]:
# Blank cell for free use. Add additional cells with Menu > Insert.
# Change the cell type with Cell > Cell Type.
# Delete a cell with Edit > Delete Cells.

---
## 5. Apply filters with VariantFiltration and evaluate results

Note you will lose all functions and variables stored in R when you switch the kernel. If you wish to avoid this, perform these analyses locally. Instructions for this section are recapitulated in the last section of the _GATK Workshop 2b. Germline Hard Filtering Tutorial_ worksheet

<span style="color:blue">Change the kernel to Python 3. Go to the menubar and select _Kernel > Change Kernel > Python 3_.</span>

### A. Filter on QUAL and tabulate baseline concordance

Based on the plots we generated, we're going to apply some filters to weed out false positives. To illustrate how VariantFiltration works, and to establish baseline performance, we first filter on QUAL < 30. By default, GATK GenotypeGVCFs filters out variants with QUAL < 10. This step produces a VCF with all the original variants; those that failed the filter are annotated with the filter name in the FILTER column.


In [0]:
# Filter callset on one annotation, QUAL < 30
! gatk VariantFiltration \
-R gs://gatk-tutorials/workshop_1702/variant_discovery/data/ref/ref.fasta \
-V /home/jupyter-user/motherSNP.vcf.gz \
--filter-expression "QUAL < 30" \
--filter-name "qual30" \
-O /home/jupyter-user/motherSNPqual30.vcf.gz

In [0]:
# Peruse the results; try adding 'grep "qual30"'
! zcat /home/jupyter-user/motherSNPqual30.vcf.gz | grep -v '##' | head -n10

In [0]:
# Calculate concordance metrics using GATK4 BETA tool Concordance
! gatk Concordance \
-eval /home/jupyter-user/motherSNPqual30.vcf.gz \
-truth gs://gatk-tutorials/workshop_1702/variant_discovery/data/resources/motherGIABsnps.vcf.gz \
-L gs://gatk-tutorials/workshop_1702/variant_discovery/data/intervals/motherHighconf.bed \
-S /home/jupyter-user/motherSNPqual30.txt
        
# View the results
! echo ""
! cat /home/jupyter-user/motherSNPqual30.txt

### B. Filter on multiple annotations simultaneously using VariantFiltration

To filter on multiple expressions, provide each in separate expression. For INFO level annotations, the parameter is  `-filter`, which should be immediately followed by the corresponding `–-filter-name` label. Here we show basic hard-filtering thresholds.

- If an annotation is missing, VariantFiltration skips any judgement on that annotation. To conservatively fail such missing annotation sites, set the `--missing-values-evaluate-as-failing` flag. 
- To filter based on FORMAT level annotations, use `--genotype-filter-expression` and `--genotype-filter-name`. 

In [0]:
# Filter callset on multiple annotations.
# Iterate on thresholds to improve precision while maintaining high sensitivity.
! gatk VariantFiltration \
-V /home/jupyter-user/motherSNP.vcf.gz \
-filter "QD < 2.0" --filter-name "QD2" \
-filter "QUAL < 30.0" --filter-name "QUAL30" \
-filter "SOR > 3.0" --filter-name "SOR3" \
-filter "FS > 60.0" --filter-name "FS60" \
-filter "MQ < 40.0" --filter-name "MQ40" \
-filter "MQRankSum < -12.5" --filter-name "MQRankSum-12.5" \
-filter "ReadPosRankSum < -8.0" --filter-name "ReadPosRankSum-8" \
-O /home/jupyter-user/motherSNPfilters.vcf.gz

In [0]:
# Sanity-check that filtering is as expected by examining filtered records and PASS records.
! zcat /home/jupyter-user/motherSNPfilters.vcf.gz | grep -v '##' | grep -v 'PASS' | head -n20 | cut -f6-10
! zcat /home/jupyter-user/motherSNPfilters.vcf.gz | grep -v '#' | grep 'PASS' | head | cut -f6-10

In [0]:
# Calculate concordance metrics using GATK4 BETA tool Concordance
! gatk Concordance \
-eval /home/jupyter-user/motherSNPfilters.vcf.gz \
-truth gs://gatk-tutorials/workshop_1702/variant_discovery/data/resources/motherGIABsnps.vcf.gz \
-L gs://gatk-tutorials/workshop_1702/variant_discovery/data/intervals/motherHighconf.bed \
-S /home/jupyter-user/motherSNPfilters.txt
        
# View the results
! echo ""
! cat /home/jupyter-user/motherSNPfilters.txt

---

We performed hard-filtering to learn about germline variant annotations. Remember that GATK recommends _Variant Quality Score Recalibration_ (VQSR) for germline variant callset filtering. For more complex variant filtering and annotation, see the Broad [Hail.is](https://hail.is/index.html) framework. 